In [1]:
import pandas as pd
from datetime import datetime, timedelta
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']


## Courses Between 50 and 20 dollars

In [5]:
min_price = 50
max_price = 200

try:
    courses = db.Courses.find(
        {"price": {"$gte": min_price, "$lte": max_price}},
        {"_id": 0, "courseId": 1, "title": 1, "price": 1}
    ).sort("price", -1)

    courses_list = list(courses)

    if courses_list:
        print(f"Courses priced between ${min_price} and ${max_price}:\n")
        for course in courses_list:
            print(f"Course ID: {course['courseId']} | Price: ${course['price']} | Title: {course['title']}")
    else:
        print("No courses found in the specified price range.")

except Exception as e:
    print(f"Error fetching courses by price range: {e}")

Courses priced between $50 and $200:

Course ID: CRS003 | Price: $148.82 | Title: Full-Stack Web Development
Course ID: CRS002 | Price: $133.25 | Title: Introduction to Data Science
Course ID: CRS008 | Price: $111.76 | Title: Serverless Applications on Cloud
Course ID: CRS005 | Price: $95.33 | Title: Serverless Applications on Cloud
Course ID: CRS001 | Price: $95.23 | Title: DevOps Fundamentals
Course ID: CRS004 | Price: $65.03 | Title: Cloud Computing with AWS
Course ID: CRS007 | Price: $63.21 | Title: Serverless Applications on Cloud
Course ID: CRS006 | Price: $57.21 | Title: Frontend Development with React


## Courses in the last 6 months 

In [4]:

months = 6

try:
    # Calculate cutoff date
    cutoff_date = datetime.now() - timedelta(days=months * 30)

    # Fetch all users
    users = list(db.Users.find(
        {},
        {"_id": 0, "userId": 1, "firstName": 1, "lastName": 1, "dateJoined": 1}
    ))

    # Filter users who joined in the last 6 months
    recent_users = [u for u in users if u.get("dateJoined") and u["dateJoined"] >= cutoff_date]

    # Sort them by dateJoined (most recent first)
    recent_users.sort(key=lambda x: x["dateJoined"], reverse=True)

    if recent_users:
        print(f"Users who joined in the last {months} months:\n")
        for user in recent_users:
            print(f"{user['userId']} - {user['firstName']} {user['lastName']} | Joined: {user['dateJoined'].strftime('%Y-%m-%d')}")
    else:
        print(f"No users joined in the last {months} months.")

except Exception as e:
    print(f"Error while fetching recent users: {e}")

Users who joined in the last 6 months:

STU008 - Alan Evans | Joined: 2025-09-20
STU006 - Eric Mcfarland | Joined: 2025-09-11
STU011 - Anthony Washington | Joined: 2025-06-18
STU012 - Colton Wilkinson | Joined: 2025-06-16
STU014 - Steven Velazquez | Joined: 2025-05-31
STU009 - Audrey Watson | Joined: 2025-04-28
STU002 - Sarah Duran | Joined: 2025-04-08


## Courses with Specific Tags

In [7]:
tags_to_search = ["python", "Data Analysis"]

try:
    # Find courses that contain any of the specified tags
    courses_with_tags = db.Courses.find(
        {"tags": {"$in": tags_to_search}},
        {"_id": 0, "courseId": 1, "title": 1, "tags": 1}
    )

    # Convert to list
    courses_list = list(courses_with_tags)

    if courses_list:
        print(f"Courses with tags {tags_to_search}:\n")
        for course in courses_list:
            print(f"{course['courseId']} - {course['title']} | Tags: {', '.join(course['tags'])}")
    else:
        print(f"No courses found with the specified tags: {tags_to_search}")

except Exception as e:
    print(f"Error fetching courses with tags: {e}")

Courses with tags ['python', 'Data Analysis']:

CRS001 - DevOps Fundamentals | Tags: Data Analysis, AI
CRS002 - Introduction to Data Science | Tags: Data Analysis, Web Dev, AI
CRS003 - Full-Stack Web Development | Tags: Python, Cloud, Data Analysis
CRS004 - Cloud Computing with AWS | Tags: Python, Data Analysis
CRS005 - Serverless Applications on Cloud | Tags: Cloud, Data Analysis, AI, Data Science, Python
CRS006 - Frontend Development with React | Tags: Python, Data Analysis


## Assignmnet Due in 7 days 

In [9]:
try:
    # Define time range (today → next 7 days)
    now = datetime.now()
    end_date = now + timedelta(days=7)

    # Query assignments within the date range
    upcoming_assignments = db.Assignments.find(
        {"dueDate": {"$gte": now, "$lte": end_date}},
        {"_id": 0, "assignmentId": 1, "title": 1, "dueDate": 1}
    )

    found = False
    for assignment in upcoming_assignments:
        print(f"- {assignment['title']} (Due: {assignment['dueDate']})")
        found = True

    if not found:
        print("No assignments are due in the next 7 days.")

except Exception as e:
    print(f"An error occurred while fetching upcoming assignments: {e}")

No assignments are due in the next 7 days.


## Enrollment per course

In [20]:
try:
    # Aggregate enrollments per course
    enrollments_per_course = db.Enrollments.aggregate([
        {
            "$group": {
                "_id": "$courseId",
                "TotalEnrollments": {"$sum": 1}
            }
        },
        {
            "$lookup": {
                "from": "Courses",
                "localField": "_id",
                "foreignField": "courseId",
                "as": "course"
            }
        },
        {"$unwind": "$course"},
        {
            "$project": {
                "_id": 0,
                "CourseId": "$_id",
                "Title": "$course.title",
                "TotalEnrollments": 1
            }
        },
        {"$sort": {"TotalEnrollments": -1}}
    ])

    
    found = False
    for record in enrollments_per_course:
        print(f"- {record['Title']} ({record['CourseId']}): {record['TotalEnrollments']} enrollment(s)")
        found = True

    if not found:
        print("No enrollment data found.")

except Exception as e:
    print(f" Error fetching enrollments per course: {e}")

- Frontend Development with React (CRS006): 3 enrollment(s)
- DevOps Fundamentals (CRS001): 3 enrollment(s)
- Serverless Applications on Cloud (CRS008): 2 enrollment(s)
- Full-Stack Web Development (CRS003): 2 enrollment(s)
- Serverless Applications on Cloud (CRS005): 1 enrollment(s)
- Cloud Computing with AWS (CRS004): 1 enrollment(s)
- Introduction to Data Science (CRS002): 1 enrollment(s)
- Serverless Applications on Cloud (CRS007): 1 enrollment(s)


## Average Course Rating

In [19]:
try:
    avg_course_rating = db.Submissions.aggregate([
        {
            "$group": {
                "_id": "$courseId",
                "AverageRating": {"$avg": "$grade"}
            }
        },
        {
            "$lookup": {
                "from": "Courses",
                "localField": "_id",
                "foreignField": "courseId",
                "as": "course"
            }
        },
        {"$unwind": "$course"},
        {
            "$project": {
                "_id": 0,
                "CourseId": "$_id",
                "Title": "$course.title",
                "AverageRating": {"$round": ["$AverageRating", 2]}
            }
        },
        {"$sort": {"AverageRating": -1}}
    ])

    results = list(avg_course_rating)

    if results:
        print("Average course rating per course:\n")
        for r in results:
            print(f"{r['Title']} ({r['CourseId']}): {r['AverageRating']}")
    else:
        print("No rating data found.")

except Exception as e:
    print(f"Error fetching average course rating: {e}")

Average course rating per course:

Full-Stack Web Development (CRS003): 88.5
Serverless Applications on Cloud (CRS007): 81.0
Cloud Computing with AWS (CRS004): 73.0
Frontend Development with React (CRS006): 59.0
Introduction to Data Science (CRS002): None
DevOps Fundamentals (CRS001): None
Serverless Applications on Cloud (CRS008): None


## Average Course by Category

In [18]:
try:
    avg_rating_by_category = db.Courses.aggregate([
        {
            "$lookup": {
                "from": "Submissions",
                "localField": "courseId",
                "foreignField": "courseId",
                "as": "grades"
            }
        },
        {"$unwind": "$grades"},
        {
            "$group": {
                "_id": "$category",
                "AverageRating": {"$avg": "$grades.grade"},
                "TotalCourses": {"$sum": 1}
            }
        },
        {
            "$project": {
                "_id": 0,
                "Category": "$_id",
                "AverageRating": {"$round": ["$AverageRating", 2]},
                "TotalCourses": 1
            }
        },
        {"$sort": {"AverageRating": -1}}
    ])

    results = list(avg_rating_by_category)

    if results:
        
        for r in results:
            print(f"Category: {r['Category']}, Total Courses: {r['TotalCourses']}, Average Rating: {r['AverageRating']}")
    else:
        print("No rating data found by category.")

except Exception as e:
    print(f"Error fetching rating by category: {e}")

Category: Web Development, Total Courses: 6, Average Rating: 78.67
Category: Cloud Computing, Total Courses: 4, Average Rating: 77.0
Category: Data Science, Total Courses: 2, Average Rating: None


## Average Grade per Student 

In [21]:
try:
    avg_grade_per_student = db.Submissions.aggregate([
        {
            "$group": {
                "_id": "$studentId",
                "averageGrade": {"$avg": "$grade"}
            }
        },
        {"$sort": {"averageGrade": -1}},
        {
            "$lookup": {
                "from": "Users",   # use correct capitalization
                "localField": "_id",
                "foreignField": "userId",
                "as": "student"
            }
        },
        {"$unwind": "$student"},
        {
            "$project": {
                "_id": 0,
                "FirstName": "$student.firstName",
                "LastName": "$student.lastName",
                "Email": "$student.email",
                "AverageGrade": {"$round": ["$averageGrade", 2]}
            }
        }
    ])

    results = list(avg_grade_per_student)

    if results:
        print("Average Grade per Student:\n")
        for r in results:
            print(f"{r['FirstName']} {r['LastName']} ({r['Email']}) — Average Grade: {r['AverageGrade']}")
    else:
        print("No grade data found for students.")

except Exception as e:
    print(f"Error fetching average grade per student: {e}")

Average Grade per Student:

Sarah Duran (haleypatterson@example.net) — Average Grade: 92.0
Steven Velazquez (ztrujillo@example.org) — Average Grade: 85.0
Colton Wilkinson (maria70@example.net) — Average Grade: 81.0
Lori Hernandez (robert66@example.net) — Average Grade: 73.0
Nancy Baldwin (farleykristen@example.org) — Average Grade: 59.0
Fernando Bowman (kreyes@example.net) — Average Grade: None
Audrey Watson (dawn87@example.net) — Average Grade: None
Mary Benson (jessewillis@example.net) — Average Grade: None
Micheal Boyer (hsmith@example.org) — Average Grade: None


## Completion Rate

In [22]:
try:
    completion_rate = db.Courses.aggregate([
        {
            "$lookup": {
                "from": "Submissions",
                "localField": "courseId",
                "foreignField": "courseId",
                "as": "submissions"
            }
        },
        {
            "$lookup": {
                "from": "Enrollments",
                "localField": "courseId",
                "foreignField": "courseId",
                "as": "enrollments"
            }
        },
        {
            "$addFields": {
                "completedCount": {
                    "$size": {
                        "$filter": {
                            "input": "$submissions",
                            "as": "s",
                            "cond": { "$eq": ["$$s.status", "graded"] }
                        }
                    }
                },
                "totalEnrolled": { "$size": "$enrollments" }
            }
        },
        {
            "$addFields": {
                "completionRate": {
                    "$cond": [
                        { "$gt": ["$totalEnrolled", 0] },
                        { "$multiply": [{ "$divide": ["$completedCount", "$totalEnrolled"] }, 100] },
                        0
                    ]
                }
            }
        },
        {
            "$project": {
                "_id": 0,
                "CourseId": "$courseId",
                "Title": "$title",
                "CompletionRate": { "$round": ["$completionRate", 2] },
                "Completed": "$completedCount",
                "TotalEnrolled": "$totalEnrolled"
            }
        },
        { "$sort": { "CompletionRate": -1 } }
    ])

    results = list(completion_rate)

    if results:
        print("Completion Rate per Course:\n")
        for r in results:
            print(f"{r['Title']} ({r['CourseId']}) — {r['Completed']} of {r['TotalEnrolled']} completed ({r['CompletionRate']}%)")
    else:
        print("No completion data found.")

except Exception as e:
    print(f"Error calculating completion rate: {e}")

Completion Rate per Course:

Serverless Applications on Cloud (CRS007) — 1 of 1 completed (100.0%)
Full-Stack Web Development (CRS003) — 2 of 2 completed (100.0%)
Cloud Computing with AWS (CRS004) — 1 of 1 completed (100.0%)
Frontend Development with React (CRS006) — 1 of 3 completed (33.33%)
Serverless Applications on Cloud (CRS005) — 0 of 1 completed (0.0%)
Introduction to Data Science (CRS002) — 0 of 1 completed (0.0%)
DevOps Fundamentals (CRS001) — 0 of 3 completed (0.0%)
Serverless Applications on Cloud (CRS008) — 0 of 2 completed (0.0%)


## Top performing STUDENT

In [23]:
top_n = 5

try:
    top_students = db.Submissions.aggregate([
        {
            "$group": {
                "_id": "$studentId",
                "averageGrade": {"$avg": "$grade"}
            }
        },
        {"$sort": {"averageGrade": -1}},
        {"$limit": top_n},
        {
            "$lookup": {
                "from": "Users",
                "localField": "_id",
                "foreignField": "userId",
                "as": "student"
            }
        },
        {"$unwind": "$student"},
        {
            "$project": {
                "_id": 0,
                "FirstName": "$student.firstName",
                "LastName": "$student.lastName",
                "Email": "$student.email",
                "AverageGrade": {"$round": ["$averageGrade", 2]}
            }
        }
    ])

    results = list(top_students)

    if results:
        print(f"Top {top_n} Students by Average Grade:\n")
        for r in results:
            print(f"{r['FirstName']} {r['LastName']} ({r['Email']}) — Average Grade: {r['AverageGrade']}")
    else:
        print("No student grade data found.")

except Exception as e:
    print(f"Error fetching top students: {e}")

Top 5 Students by Average Grade:

Sarah Duran (haleypatterson@example.net) — Average Grade: 92.0
Steven Velazquez (ztrujillo@example.org) — Average Grade: 85.0
Colton Wilkinson (maria70@example.net) — Average Grade: 81.0
Lori Hernandez (robert66@example.net) — Average Grade: 73.0
Nancy Baldwin (farleykristen@example.org) — Average Grade: 59.0


## Most Popular Course Category

In [24]:
try:
    popular_categories = db.Enrollments.aggregate([
        {
            "$lookup": {
                "from": "Courses",
                "localField": "courseId",
                "foreignField": "courseId",
                "as": "course"
            }
        },
        {"$unwind": "$course"},
        {
            "$group": {
                "_id": "$course.category",
                "totalEnrollments": {"$sum": 1}
            }
        },
        {"$sort": {"totalEnrollments": -1}}
    ])

    results = list(popular_categories)

    if results:
        print("Most Popular Course Categories:\n")
        for r in results:
            print(f"Category: {r['_id']}, Total Enrollments: {r['totalEnrollments']}")
    else:
        print("No enrollment data found.")

except Exception as e:
    print(f"Error fetching popular course categories: {e}")

Most Popular Course Categories:

Category: Cloud Computing, Total Enrollments: 8
Category: Web Development, Total Enrollments: 5
Category: Data Science, Total Enrollments: 1


## Monthly Enrollment

In [ ]:

try:
    # Aggregate monthly enrollment trends
    monthly_trends = db.Enrollments.aggregate([
        {
            "$group": {
                "_id": {"year": {"$year": "$enrolledAt"}, "month": {"$month": "$enrolledAt"}},
                "totalEnrollments": {"$sum": 1}
            }
        },
        {"$sort": {"_id.year": 1, "_id.month": 1}}
    ])

    trends_list = list(monthly_trends)

    if trends_list:
        print("Monthly Enrollment Trends:\n")
        for item in trends_list:
            year = item['_id']['year']
            month = item['_id']['month']
            total = item['totalEnrollments']
            print(f"Year: {year}, Month: {month}, TotalEnrollments: {total}")
    else:
        print("No enrollment data found.")

except Exception as e:
    print(f"Error fetching monthly enrollment trends: {e}")

Monthly Enrollment Trends:

Year: 2024, Month: 10, TotalEnrollments: 2
Year: 2024, Month: 12, TotalEnrollments: 1
Year: 2025, Month: 1, TotalEnrollments: 2
Year: 2025, Month: 2, TotalEnrollments: 1
Year: 2025, Month: 4, TotalEnrollments: 2
Year: 2025, Month: 6, TotalEnrollments: 1
Year: 2025, Month: 7, TotalEnrollments: 1
Year: 2025, Month: 8, TotalEnrollments: 2
Year: 2025, Month: 9, TotalEnrollments: 2


## Total Submission

In [ ]:

try:
    # Aggregate total submissions per student
    engagement_metrics = db.Submissions.aggregate([
        {
            "$group": {
                "_id": "$studentId",
                "totalSubmissions": {"$sum": 1}
            }
        },
        {"$sort": {"totalSubmissions": -1}}
    ])

    # Convert to list and print directly
    engagement_list = list(engagement_metrics)

    if engagement_list:
        print("Student Engagement Metrics:\n")
        for item in engagement_list:
            print(f"StudentID: {item['_id']}, TotalSubmissions: {item['totalSubmissions']}")
    else:
        print("No submission data found.")

except Exception as e:
    print(f"Error fetching student engagement metrics: {e}")

Student Engagement Metrics:

StudentID: STU003, TotalSubmissions: 3
StudentID: STU010, TotalSubmissions: 2
StudentID: STU005, TotalSubmissions: 1
StudentID: STU014, TotalSubmissions: 1
StudentID: STU015, TotalSubmissions: 1
StudentID: STU012, TotalSubmissions: 1
StudentID: STU009, TotalSubmissions: 1
StudentID: STU002, TotalSubmissions: 1
StudentID: STU004, TotalSubmissions: 1
